In [8]:
import datetime, itertools, sys
import pandas as ps
from subprocess import Popen, PIPE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from pymystem3 import Mystem

In [9]:
data = ps.read_csv("data/spelled-f.csv", sep=';', header=None,
                   index_col=0,names=['id','title','text','cluster','date','publisher'])

## Предварительная очистка данных

In [10]:
data = data[~data["cluster"].isin(["-", "S", "Standard "])]
print("Число записей в таблице:", len(data))

Число записей в таблице: 32317


In [11]:
day_counts = {}
for _, row in data.iterrows():
    day_counts[row["date"]] = day_counts.get(row["date"], 0) + 1
max_date = max((count, date) for date, count in day_counts.items())[1]
print("Самый плотный день - {}, число записей: {}".format(max_date, day_counts[max_date]))

Самый плотный день - 2016-01-27, число записей: 19846


In [12]:
appropriate_days = [date for date, count in day_counts.items() if 520 <= count <= 530]
ch_day = appropriate_days[0]
data_1d = data[data["date"] == ch_day]
print("Выбранный день - {}, число записей: {}".format(ch_day, day_counts[ch_day]))

Выбранный день - 2015-05-30, число записей: 523


## Лемматизация текстов

In [16]:
m = Mystem()
cluster_index = {cluster: i for i, cluster in enumerate(data["cluster"].unique())}
clusters = [cluster_index[row["cluster"]] for _, row in data_1d.iterrows()]
messages = []
must_have = ["S", "V", "A"]
for _, row in data_1d.iterrows():
    text = row["title"] + ". " + row["text"]
    words = []
    for info in m.analyze(text):
        try:
            an = info["analysis"][0]
        except (KeyError, IndexError):
            continue
        if any(t in an["gr"] for t in must_have):
            words.append(an["lex"])
    messages.append(" ".join(words))

In [17]:
print(messages[0])
print(len(messages), len(clusters))

памятный комплекс чкаловец герой отечество открывать воронежский область многий воронежец внимательно следить соревнование летчик полигон погоновый мы открывать памятник герой советский союз дважды герой ссср герой россия выпускник
523 523


## Построение матрицы TF*IDF

In [18]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(messages)
print(tfidf_matrix.shape)

(523, 3513)


## Эксперимент с косинусовой мерой

In [19]:
index_cluster = {index: cluster for cluster, index in cluster_index.items()}
cluster_counts = {}
for cluster in clusters:
    cluster_counts[cluster] = cluster_counts.get(cluster, 0) + 1
max_cluster = max((count, cluster) for cluster, count in cluster_counts.items())[1]
print("Самый большой кластер - '{}' ({})\nЧисленность - {}".format(
        index_cluster[max_cluster], max_cluster, cluster_counts[max_cluster]))

Самый большой кластер - 'В Тюмени нарушитель угнал машину ДПС, пока оформлялся протокол' (1706)
Численность - 3


In [20]:
print("Все сообщения кластера:\n")
for i, cluster in enumerate(clusters):
    if cluster == max_cluster:
        print("  " + messages[i])

Все сообщения кластера:

  тюмень пассажир останавливать лад время составление протокол угнать машина тот момент когда инспектор оформлять протокол задержание водитель останавливать лад ее пассажир воспользоваться момент попытаться скрываться отечественный машина тот
  тюмень нарушитель угнать машина силовик составлять протокол силовик останавливать тонировать лад модель салон сидеть пьяный молодой человек который препятствовать составление протокол
  угон полицейский машина тюмень оказываться утка представитель гибдд тюменский область опровергать информация угон автомобиль время составление административный протокол сегодня утро улица ватутин тюмень наряд быть


In [21]:
target_messages = [i for i, cluster in enumerate(clusters) if cluster == max_cluster]
ind = target_messages[0]
sim_factors = cosine_similarity(tfidf_matrix[ind:ind+1], tfidf_matrix)
for i in target_messages:
    print(sim_factors[0][i])

1.0
0.392630469105
0.207949674337


In [22]:
th_values = (0.8, 0.6, 0.4, 0.35, 0.3, 0.25, 0.2)
thresholds = {t: 0 for t in th_values}
for sf in sim_factors[0]:
    for th in th_values:
        if sf >= th:
            thresholds[th] += 1
print(sorted(thresholds.items()))

[(0.2, 3), (0.25, 2), (0.3, 2), (0.35, 2), (0.4, 1), (0.6, 1), (0.8, 1)]


## Эксперимент на полноразмерных данных

In [23]:
sim_factors_full = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [24]:
def get_expected_and_predicted(sim_factors, clusters, th=0.25):
    res = {"pair": [], "y": [], "p": []}
    for i1, i2 in itertools.combinations(range(len(sim_factors)), 2):
        res["pair"].append(sorted((i1, i2)))
        res["y"].append(clusters[i1] == clusters[i2])
        res["p"].append(sim_factors[i1][i2] >= th)
    return ps.DataFrame(res, index=None)

In [25]:
def cross_class_report(res):
    classes = res["y"].unique()
    table = ps.DataFrame(index=classes, columns=classes)
    for true_cls in classes:
        tmp = res[res["y"] == true_cls]
        for pred_cls in classes:
            table[pred_cls][true_cls] = len(tmp[tmp["p"] == pred_cls])
    return table

In [26]:
th_values = [0.25 + 0.01*i for i in range(11)]
for th in th_values:
    res = get_expected_and_predicted(sim_factors_full, clusters, th)
    print("*** THRESHOLD: {} ***".format(th))
    print(classification_report(res["y"], res["p"]))
    print(cross_class_report(res), "\n\n")

*** THRESHOLD: 0.25 ***
             precision    recall  f1-score   support

      False       1.00      1.00      1.00    136121
       True       0.39      0.77      0.52       382

avg / total       1.00      1.00      1.00    136503

        False True 
False  135654   467
True       87   295 


*** THRESHOLD: 0.26 ***
             precision    recall  f1-score   support

      False       1.00      1.00      1.00    136121
       True       0.40      0.76      0.52       382

avg / total       1.00      1.00      1.00    136503

        False True 
False  135684   437
True       93   289 


*** THRESHOLD: 0.27 ***
             precision    recall  f1-score   support

      False       1.00      1.00      1.00    136121
       True       0.41      0.73      0.52       382

avg / total       1.00      1.00      1.00    136503

        False True 
False  135719   402
True      103   279 


*** THRESHOLD: 0.28 ***
             precision    recall  f1-score   support

      False     